Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import matplotlib.pyplot as plt
import numpy as np
import os
import tarfile
import urllib
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
import cPickle as pickle

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [2]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print 'Found and verified', filename
  else:
    raise Exception(
      'Failed to verify' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [3]:
num_classes = 10

def extract(filename):
  tar = tarfile.open(filename)
  tar.extractall()
  tar.close()
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print data_folders
  return data_folders
  
train_folders = extract(train_filename)
test_folders = extract(test_filename)

['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [4]:
a_dir = train_folders[0]
one_example = os.listdir(a_dir)[100]
Image(filename=a_dir + "/" + one_example)

Now let's load the data in a more manageable format.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. The labels will be stored into a separate array of integers 0 through 9.

A few images might not be readable, we'll just skip them.

In [5]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load(data_folders, min_num_images, max_num_images):
  dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size), dtype=np.float32)
  labels = np.ndarray(shape=(max_num_images), dtype=np.int32)
  label_index = 0
  image_index = 0
  for folder in data_folders:
    print folder
    for image in os.listdir(folder):
      if image_index >= max_num_images:
        raise Exception('More images than expected: %d >= %d' % (
          num_images, max_num_images))
      image_file = os.path.join(folder, image)
      try:
        image_data = (ndimage.imread(image_file).astype(float) -
                      pixel_depth / 2) / pixel_depth
        if image_data.shape != (image_size, image_size):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))
        dataset[image_index, :, :] = image_data
        labels[image_index] = label_index
        image_index += 1
      except IOError as e:
        print 'Could not read:', image_file, ':', e, '- it\'s ok, skipping.'
    label_index += 1
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  labels = labels[0:num_images]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' % (
        num_images, min_num_images))
  print 'Full dataset tensor:', dataset.shape
  print 'Mean:', np.mean(dataset)
  print 'Standard deviation:', np.std(dataset)
  print 'Labels:', labels.shape
  return dataset, labels
train_dataset, train_labels = load(train_folders, 450000, 550000)
test_dataset, test_labels = load(test_folders, 18000, 20000)

notMNIST_large/A
Could not read: notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : cannot identify image file 'notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png' - it's ok, skipping.
Could not read: notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : cannot identify image file 'notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png' - it's ok, skipping.
Could not read: notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : cannot identify image file 'notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png' - it's ok, skipping.
notMNIST_large/B
Could not read: notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png : cannot identify image file 'notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png' - it's ok, skipping.
notMNIST_large/C
notMNIST_large/D
Could not read: notMNIST_large/D/VHJhbnNpdCBCb2xkLnR0Zg==.png : cannot identify image file 'notMNIST_large/D/VHJhbnNpdCBCb2xkLnR0Zg==.png' - it's ok, skipping.
notMNIST_large/E
notMNIST_large/F
notMNIST_large/G
notMNIST_large/H


---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [6]:
random_idx = 100
random_data = train_dataset[random_idx]
random_label = train_labels[random_idx]
print random_label
plt.imshow(random_data)
plt.show()

0


Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [7]:
np.random.seed(133)
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

---
Problem 3
---------
Convince yourself that the data is still good after shuffling!

---

In [8]:
random_idx = 100
random_data = train_dataset[random_idx]
random_label = train_labels[random_idx]
print random_label
plt.imshow(random_data)
plt.show()

2


---
Problem 4
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [9]:
train_labels
np.histogram(train_labels)

(array([52909, 52911, 52912, 52911, 52912, 52912, 52912, 52912, 52912, 52911]),
 array([ 0. ,  0.9,  1.8,  2.7,  3.6,  4.5,  5.4,  6.3,  7.2,  8.1,  9. ]))

Prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune train_size as needed.

Also create a validation dataset for hyperparameter tuning.

In [10]:
train_size = 200000
valid_size = 10000

valid_dataset = train_dataset[:valid_size,:,:]
valid_labels = train_labels[:valid_size]
train_dataset = train_dataset[valid_size:valid_size+train_size,:,:]
train_labels = train_labels[valid_size:valid_size+train_size]
print 'Training', train_dataset.shape, train_labels.shape
print 'Validation', valid_dataset.shape, valid_labels.shape

Training (200000, 28, 28) (200000,)
Validation (10000, 28, 28) (10000,)


Finally, let's save the data for later reuse:

In [11]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print 'Unable to save data to', pickle_file, ':', e
  raise

In [12]:
statinfo = os.stat(pickle_file)
print 'Compressed pickle size:', statinfo.st_size

Compressed pickle size: 718193801


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.
Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [15]:
save = pickle.load(open(pickle_file))
np.unique(save['train_dataset'])

array([-0.5       , -0.49607843, -0.49215686, -0.48823529, -0.48431373,
       -0.48039216, -0.47647059, -0.47254902, -0.46862745, -0.46470588,
       -0.46078432, -0.45686275, -0.45294118, -0.44901961, -0.44509804,
       -0.44117647, -0.43725491, -0.43333334, -0.42941177, -0.4254902 ,
       -0.42156863, -0.41764706, -0.4137255 , -0.40980393, -0.40588236,
       -0.40196079, -0.39803922, -0.39411765, -0.39019608, -0.38627452,
       -0.38235295, -0.37843138, -0.37450981, -0.37058824, -0.36666667,
       -0.36274511, -0.35882354, -0.35490197, -0.3509804 , -0.34705883,
       -0.34313726, -0.3392157 , -0.33529413, -0.33137256, -0.32745099,
       -0.32352942, -0.31960785, -0.31568629, -0.31176472, -0.30784315,
       -0.30392158, -0.30000001, -0.29607844, -0.29215688, -0.28823531,
       -0.28431374, -0.28039217, -0.2764706 , -0.27254903, -0.26862746,
       -0.2647059 , -0.26078433, -0.25686276, -0.25294119, -0.24901961,
       -0.24509804, -0.24117647, -0.2372549 , -0.23333333, -0.22

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [16]:
train_dataset_unroll = np.reshape(train_dataset, (train_dataset.shape[0], train_dataset.shape[1] * train_dataset.shape[2]))
valid_dataset_unroll = np.reshape(valid_dataset, (valid_dataset.shape[0], valid_dataset.shape[1] * valid_dataset.shape[2]))

for reg_term in [1., .1, .3, .01, 3, 10.]:
    print "Reg term %.2f" % reg_term
    for batch_size in [50, 100, 1000, 5000]:
        model = LogisticRegression(C=reg_term, n_jobs=-1)
        model.fit(train_dataset_unroll[:batch_size], train_labels[:batch_size])
        train_score = model.score(train_dataset_unroll[:batch_size], train_labels[:batch_size])
        valid_score = model.score(valid_dataset_unroll, valid_labels)
        print "Train accuracy on %d: %.2f" % (batch_size, train_score)
        print "Valid accuracy on %d: %.2f" % (batch_size, valid_score)


Reg term 1.00
Train accuracy on 50: 1.00
Valid accuracy on 50: 0.58
Train accuracy on 100: 1.00
Valid accuracy on 100: 0.68
Train accuracy on 1000: 0.99
Valid accuracy on 1000: 0.76
Train accuracy on 5000: 0.94
Valid accuracy on 5000: 0.77
Reg term 0.10
Train accuracy on 50: 1.00
Valid accuracy on 50: 0.58
Train accuracy on 100: 1.00
Valid accuracy on 100: 0.68
Train accuracy on 1000: 0.94
Valid accuracy on 1000: 0.78
Train accuracy on 5000: 0.88
Valid accuracy on 5000: 0.80
Reg term 0.30
Train accuracy on 50: 1.00
Valid accuracy on 50: 0.58
Train accuracy on 100: 1.00
Valid accuracy on 100: 0.68
Train accuracy on 1000: 0.98
Valid accuracy on 1000: 0.77
Train accuracy on 5000: 0.91
Valid accuracy on 5000: 0.79
Reg term 0.01
Train accuracy on 50: 0.98
Valid accuracy on 50: 0.52
Train accuracy on 100: 0.94
Valid accuracy on 100: 0.64
Train accuracy on 1000: 0.86
Valid accuracy on 1000: 0.79
Train accuracy on 5000: 0.83
Valid accuracy on 5000: 0.81
Reg term 3.00
Train accuracy on 50: 1.00

In [ ]:
reg_term = 0.01
batch_size = 5000
model = LogisticRegression(C=reg_term, n_jobs=-1)
model.fit(train_dataset_unroll[:batch_size], train_labels[:batch_size])
test_dataset_unroll = np.reshape(test_dataset, (test_dataset.shape[0], test_dataset.shape[1] * test_dataset.shape[2]))
test_score = model.score(test_dataset_unroll, test_labels)
print "Test accuracy on %d data with %.2f reg term: %.2f" % (batch_size, reg_term, test_score)